<a href="https://colab.research.google.com/github/ashikshafi08/Learning_Tensorflow/blob/main/Straight%20From%20Docs/Text_Classification_Docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the needed packages 
import tensorflow as tf 
import os 
import re 
import shutil 
import string 

# From the tensorflow 
from tensorflow.keras import layers 
from tensorflow.keras import preprocessing 
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Version
print(tf.__version__)

2.5.0


### Sentiment analysis 

This notebook trains a sentiment analysis model to classify movie reviews as positive or negative, based on the text review

In [2]:
# Downloading the datasets 
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1" , 
                                  url , 
                                  untar = True , cache_dir = '.' , 
                                  cache_subdir = '')
dataset_dir = os.path.join(os.path.dirname(dataset) , "aclImdb")

84131840/84125825 [==============================] - 6s 0us/step


In [3]:
# Looking whats inside the directory 
os.listdir(dataset_dir)

['imdb.vocab', 'train', 'README', 'imdbEr.txt', 'test']

In [4]:
train_dir = os.path.join(dataset_dir , 'train')
os.listdir(train_dir)

['unsup',
 'urls_pos.txt',
 'urls_neg.txt',
 'pos',
 'labeledBow.feat',
 'urls_unsup.txt',
 'neg',
 'unsupBow.feat']

`aclImdb/train/pos` and `aclImdb/train/neg` directories contain many text files, each of is a movie review. 

In [5]:
sample_file = os.path.join(train_dir , 'pos/1181_9.txt')
with open(sample_file) as f:
  print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


In [6]:
# Removing the unsup file 
remove_dir = os.path.join(train_dir , 'unsup')
shutil.rmtree(remove_dir)

In [7]:
# Loading the data and createing dataset object 
batch_size = 32 
seed = 42 

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train',
                                                                  batch_size = batch_size , 
                                                                  validation_split = 0.2 , 
                                                                  subset ='training' , 
                                                                  seed = seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [8]:
# Looking into the data 
for text_batch , label_batch in raw_train_ds.take(1):
  for i in range(3):
    print('Review' , text_batch.numpy()[i])
    print('Label', label_batch.numpy()[i])


Review b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label 0
Review b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get into 

These reviews contain raw text (with punctuation and occasional HTML tags). The labels are 0 or 1. 

In [9]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to neg
Label 1 corresponds to pos


In [10]:
# Now reading in the validation set 
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train',
                                                                  batch_size = batch_size , 
                                                                  validation_split = 0.2 , 
                                                                  subset ='validation' , 
                                                                  seed = seed)

# Loading in the test data 
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/test', 
                                                                batch_size = batch_size)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [11]:
raw_train_ds , raw_val_ds

(<BatchDataset shapes: ((None,), (None,)), types: (tf.string, tf.int32)>,
 <BatchDataset shapes: ((None,), (None,)), types: (tf.string, tf.int32)>)

Now its time to do some preprocessing on our data, where we will standardize, tokenize and vectorize the data. 


HTML tags like <br/> wont be removed by the default standardizer in `TextVectorization` layer which basically converts text to lowercase and strips punctuation by default. 

In [12]:
# Making our custom standardization function 
def custom_standardization(input_text):
  ''' Which helps to strip even some HTML tags off'''

  # Lowercasing all the text
  lowercase = tf.strings.lower(input_text)
  stripped_html = tf.strings.regex_replace(lowercase , '<br/>' , ' ')
  return tf.strings.regex_replace(stripped_html , 
                                  '[%s]' % re.escape(string.punctuation) ,  '')


`Text_Vectorization`: 
- Now we will vectorize the data 
- Set the `output_mode` to `int` to create unique indices for each token. 
- `sequence_length` so we can truncate our sequences to exactly the values in the `sequence_length`

In [13]:
# Using the text vectorization layer 
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize = custom_standardization,
    max_tokens = max_features , 
    output_mode = 'int', 
    output_sequence_length = sequence_length
)

# Fitting our training texts too the vectorizer 
train_text = raw_train_ds.map(lambda x , y: x) # grabbing only the texts (no need labels)
vectorize_layer.adapt(train_text)

In [14]:
# Simple function to vectorize the text 
def vectorize_text(text , label):
  text = tf.expand_dims(text , -1)
  return vectorize_layer(text), label

In [15]:
# Visualizing the vectors wrt the text
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'Great movie - especially the music - Etta James - "At Last". This speaks volumes when you have finally found that special someone.', shape=(), dtype=string)
Label neg
Vectorized review (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[  87,   18,  259,    2,  223,    1,  566,   31,  228,   11, 2422,
           1,   52,   23,   26,  400,  250,   12,  308,  280,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       

In [16]:
# Lookup the string that each integer corresponds to
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

1287 --->  naked
 313 --->  night
Vocabulary size: 10000


In [17]:
# Applying vectorization to all the dataset object 
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [18]:
# Optimizing for best performance 
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)
test_ds = test_ds.cache().prefetch(AUTOTUNE)

In [19]:
# Building the model 
embedding_dim = 16 

inputs = layers.Input(shape = (250,))
x = layers.Embedding(input_dim = max_features + 1 , output_dim = embedding_dim)(inputs)
x = layers.Dropout(0.2)(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1)(x)

# Packing into a model 
model = tf.keras.Model(inputs , outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 250)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 250, 16)           160016    
_________________________________________________________________
dropout (Dropout)            (None, 250, 16)           0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_______________________________________________________

The layer are stacked sequentially to build the classifier: 

- First layer is a `Embedding` layer, which takes the integer-encoded reviews and looks up an embedding vector for each word-index. The vectors add dimension to the output array. The resulting dimensions are --> `(batch_sequence , embedding)`.
- The `GlobalAveragePooling1D` layer returns a fixed length output vector for each example by averaging over the sequence dimension. 
- The fixed-length output vector is piped through a fully connected (Dense) layer with 16 hidden units. 
- Last layer acts as a output node

In [20]:
# Compiling the model 
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits= True) , 
              optimizer = tf.keras.optimizers.Adam(), 
              metrics = tf.metrics.BinaryAccuracy(threshold= 0.0))

In [21]:
# Fitting the model for 10 epochs 
epochs = 10 
history = model.fit(train_ds , 
                    validation_data = val_ds , 
                    epochs = epochs)

Epoch 1/10
625/625 [==============================] - 11s 17ms/step - loss: 0.6639 - binary_accuracy: 0.6932 - val_loss: 0.6153 - val_binary_accuracy: 0.7676
Epoch 2/10
625/625 [==============================] - 4s 6ms/step - loss: 0.5505 - binary_accuracy: 0.7988 - val_loss: 0.5005 - val_binary_accuracy: 0.8202
Epoch 3/10
625/625 [==============================] - 4s 6ms/step - loss: 0.4472 - binary_accuracy: 0.8442 - val_loss: 0.4233 - val_binary_accuracy: 0.8476
Epoch 4/10
625/625 [==============================] - 4s 6ms/step - loss: 0.3819 - binary_accuracy: 0.8650 - val_loss: 0.3772 - val_binary_accuracy: 0.8582
Epoch 5/10
625/625 [==============================] - 4s 6ms/step - loss: 0.3386 - binary_accuracy: 0.8758 - val_loss: 0.3486 - val_binary_accuracy: 0.8650
Epoch 6/10
625/625 [==============================] - 4s 6ms/step - loss: 0.3080 - binary_accuracy: 0.8872 - val_loss: 0.3293 - val_binary_accuracy: 0.8694
Epoch 7/10
625/625 [==============================] - 4s 6ms/s

In [22]:
# Evaluate the model 
loss , accuracy = model.evaluate(test_ds)
loss , accuracy

782/782 [==============================] - 9s 11ms/step - loss: 0.3138 - binary_accuracy: 0.8715


(0.3138383626937866, 0.8715199828147888)

## Exercise: Multi-class Classification on Stackoverflow Questions



In [23]:
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'

stack_dataset = tf.keras.utils.get_file('stack_overflow_16k' , 
                                        dataset_url , 
                                        untar = True , 
                                        cache_dir = '.' , 
                                        cache_subdir = '')


6053888/6053168 [==============================] - 0s 0us/step


In [27]:
# !mv '/content/train' '/content/data'
# !mv '/content/test' '/content/data'

In [28]:
# Looking whats in the train dir 
train_dir = 'data/train/'
test_dir = 'data/test/'
#/content/data/train

os.listdir(train_dir + 'python')[:10]

['1575.txt',
 '582.txt',
 '25.txt',
 '1441.txt',
 '330.txt',
 '751.txt',
 '1176.txt',
 '1087.txt',
 '1766.txt',
 '1047.txt']

In [29]:
# Getting the train and validation dataset 

train_ds = tf.keras.preprocessing.text_dataset_from_directory(train_dir , 
                                                              batch_size = batch_size , 
                                                              validation_split = 0.2 , 
                                                              subset = 'training' , 
                                                              seed = seed)

val_ds = tf.keras.preprocessing.text_dataset_from_directory(train_dir , 
                                                            batch_size = batch_size , 
                                                            subset = 'validation' , 
                                                            seed = 42 , 
                                                            validation_split = 0.2)


Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [30]:
for text , labels in train_ds.take(1):
  print(text[:2])
  print(labels[:2])

tf.Tensor(
[b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can be easily fixed, please forgive me. my program has a tester class with a main. when i send that to my regularpolygon class, it sends it to the wrong constructor. i have two constructors. 1 without perameters..public regularpolygon().    {.       mynumsides = 5;.       mysidelength = 30;.    }//end default constructor...and my second, with perameters. ..public regularpolygon(int numsides, double sidelength).    {.        mynumsides = numsides;.        mysidelength = sidelength;.    }// end constructor...in my tester class i have these two lines:..regularpolygon shape = new regularpolygon(numsides, sidelength);.        shape.menu();...numsides and sidelength were declared and initialized earlier in the testing class...so what i want to happen, is the tester class sends numsides and sidelength to the second constructor and use it in that class. but it only uses the default co

In [ ]:
tf.strings.ngrams(tf.strings.strip(text[:2]) , ngram_width=1)
tf.strings.regex_replace(text[0] , '  ', '') 
tf.strings.lower(text[0])

In [31]:
lower = tf.strings.lower(text[0])
rep_space = tf.strings.regex_replace(lower , '  ', '')
rep_space

<tf.Tensor: shape=(), dtype=string, numpy=b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can be easily fixed, please forgive me. my program has a tester class with a main. when i send that to my regularpolygon class, it sends it to the wrong constructor. i have two constructors. 1 without perameters..public regularpolygon().{. mynumsides = 5;. mysidelength = 30;.}//end default constructor...and my second, with perameters. ..public regularpolygon(int numsides, double sidelength).{.mynumsides = numsides;.mysidelength = sidelength;.}// end constructor...in my tester class i have these two lines:..regularpolygon shape = new regularpolygon(numsides, sidelength);.shape.menu();...numsides and sidelength were declared and initialized earlier in the testing class...so what i want to happen, is the tester class sends numsides and sidelength to the second constructor and use it in that class. but it only uses the default constructor, which there

In [33]:
def custom_stand(input_text):
  lower = tf.strings.lower(input_text)
  rep_space = tf.strings.regex_replace(lower , '  ', '')
  return rep_space
  

In [36]:
train_sentences = train_ds.map(lambda x,y: x)
#train_sentences = tf.expand_dims(train_sentences , -1)
train_sentences

<MapDataset shapes: (None,), types: tf.string>

In [39]:
# Creating our TextVectorization layer 
max_features = 15000 
sequence_length = 250 

multi_vectorize_layer = TextVectorization(
    standardize = custom_stand, 
    max_tokens = max_features , 
    output_mode = 'int', 
    output_sequence_length = sequence_length
)

# Fitting our training texts on our vectorize layer 
train_sentences = train_ds.map(lambda x,y: x)
multi_vectorize_layer.adapt(train_sentences)

In [40]:
# Getting our test dataset 
test_data = tf.keras.preprocessing.text_dataset_from_directory('data/test', 
                                                                batch_size = batch_size)


Found 8000 files belonging to 4 classes.


In [42]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_data.cache().prefetch(buffer_size=AUTOTUNE)


In [43]:
train_ds

<PrefetchDataset shapes: ((None,), (None,)), types: (tf.string, tf.int32)>

In [49]:
mult_model = tf.keras.Sequential([
  layers.Input(shape = (1,) , dtype = tf.string), 
  multi_vectorize_layer,
  layers.Embedding(max_features + 1 , embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(4 , activation = 'softmax')
])

mult_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_2 (TextVe (None, 250)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 250, 16)           240016    
_________________________________________________________________
dropout_6 (Dropout)          (None, 250, 16)           0         
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 68        
Total params: 240,084
Trainable params: 240,084
Non-trainable params: 0
________________________________________________

In [50]:
# Compiling the model 
mult_model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(), 
                   optimizer = tf.keras.optimizers.Adam(), 
                   metrics = ['accuracy'])


# Fit the model 
mult_model.fit(train_ds , 
               validation_data = val_ds ,
               epochs = 20)

Epoch 1/20
200/200 [==============================] - 3s 11ms/step - loss: 1.3820 - accuracy: 0.3105 - val_loss: 1.3755 - val_accuracy: 0.3881
Epoch 2/20
200/200 [==============================] - 2s 11ms/step - loss: 1.3637 - accuracy: 0.4105 - val_loss: 1.3533 - val_accuracy: 0.4931
Epoch 3/20
200/200 [==============================] - 2s 11ms/step - loss: 1.3338 - accuracy: 0.4711 - val_loss: 1.3189 - val_accuracy: 0.5319
Epoch 4/20
200/200 [==============================] - 2s 10ms/step - loss: 1.2882 - accuracy: 0.5481 - val_loss: 1.2709 - val_accuracy: 0.5863
Epoch 5/20
200/200 [==============================] - 2s 10ms/step - loss: 1.2278 - accuracy: 0.6095 - val_loss: 1.2128 - val_accuracy: 0.6375
Epoch 6/20
200/200 [==============================] - 2s 10ms/step - loss: 1.1615 - accuracy: 0.6581 - val_loss: 1.1503 - val_accuracy: 0.6650
Epoch 7/20
200/200 [==============================] - 2s 11ms/step - loss: 1.0918 - accuracy: 0.6959 - val_loss: 1.0892 - val_accuracy: 0.6881

In [51]:
# Evaluating on test data

loss, accuracy = mult_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

250/250 [==============================] - 3s 11ms/step - loss: 0.7099 - accuracy: 0.7526
Loss:  0.7098891139030457
Accuracy:  0.7526249885559082


## Text Classification with TensorFlow Hub 

In [53]:
import tensorflow_datasets as tfds

In [54]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete59WM1L/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete59WM1L/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete59WM1L/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [55]:
# Printing first 10 examples 
train_examples_batch , train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [67]:
import tensorflow_hub as hub

# Downloading the tf hub model 
url = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(url , input_shape = (1,), 
                           dtype = tf.string , trainable = True)

# Looking inside our hub layer 
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [68]:
train_data

<PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [70]:
# Building the model 
inputs = layers.Input(shape = (), dtype = tf.string)
x = hub_layer(inputs)
x = layers.Dense(16 , activation= 'relu')(x)
outputs = layers.Dense(1)(x)

# Packing into a model 
model = tf.keras.Model(inputs , outputs)

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None,)]                 0         
_________________________________________________________________
keras_layer_3 (KerasLayer)   (None, 50)                48190600  
_________________________________________________________________
dense_4 (Dense)              (None, 16)                816       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [71]:
# Compiling the model 
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['accuracy'])

# Fit the model 
history = model.fit(train_data.shuffle(10000).batch(512), 
                    epochs = 10 , 
                    validation_data = validation_data.batch(512), 
                    verbose = 1)

Epoch 1/10
30/30 [==============================] - 20s 606ms/step - loss: 0.6759 - accuracy: 0.5096 - val_loss: 0.6375 - val_accuracy: 0.5819
Epoch 2/10
30/30 [==============================] - 18s 575ms/step - loss: 0.5878 - accuracy: 0.6446 - val_loss: 0.5421 - val_accuracy: 0.7122
Epoch 3/10
30/30 [==============================] - 18s 581ms/step - loss: 0.4568 - accuracy: 0.7941 - val_loss: 0.4305 - val_accuracy: 0.8190
Epoch 4/10
30/30 [==============================] - 18s 585ms/step - loss: 0.3271 - accuracy: 0.8765 - val_loss: 0.3591 - val_accuracy: 0.8506
Epoch 5/10
30/30 [==============================] - 18s 574ms/step - loss: 0.2371 - accuracy: 0.9157 - val_loss: 0.3262 - val_accuracy: 0.8654
Epoch 6/10
30/30 [==============================] - 18s 582ms/step - loss: 0.1745 - accuracy: 0.9461 - val_loss: 0.3108 - val_accuracy: 0.8718
Epoch 7/10
30/30 [==============================] - 18s 583ms/step - loss: 0.1284 - accuracy: 0.9635 - val_loss: 0.3048 - val_accuracy: 0.8731

In [72]:
# Evaluate the model 
results = model.evaluate(test_data.batch(512) , verbose = 2)
results

49/49 - 3s - loss: 0.3545 - accuracy: 0.8548


[0.35447990894317627, 0.8547599911689758]